In [36]:
%load_ext autoreload
%autoreload 2
from mobi import Mobi

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
book = Mobi("../data/hugo_les_miserables_cosette.prc");
book.parse();

NameError: name 'xrange' is not defined

In [38]:
!pip install EbookLib

     |████████████████████████████████| 112kB 2.1MB/s eta 0:00:01
     |████████████████████████████████| 5.7MB 7.4MB/s eta 0:00:01
  Stored in directory: /home/yrieix/.cache/pip/wheels/84/11/01/951369cbbf8f96878786a1f4da68bd7ac19a5d945b38e03d54
Successfully built EbookLib


In [40]:
import ebooklib
from ebooklib import epub

book = epub.read_epub('../data/hugo_les_miserables_cosette.prc')

for image in book.get_items_of_type(ebooklib.ITEM_IMAGE):
    print(image)

EpubException: 'Bad Zip file'